# MAS - Project 

## 3 models comparison 
Comparison across brench and bound deep search first, IDA* and my genetic algorithm implementation.

Lib

In [1]:
import random
import numpy as np
import itertools
import time
import amd_project_functions as amd

## The 3 algorithms 

### B&B DSF

In [2]:
class SearchAlgorithm:
    def __init__(self, O, Theta, p, o_XY, v_XY, g, u, L = float('-inf'), CB = set()):
        """
        Initialize the SearchAlgorithm object.
        """
        self.O = O
        self.n = len(O) - 1 # check that 
        self.Theta = Theta
        self.p = p
        self.o_XY = o_XY
        self.v_XY = v_XY
        self.g = g
        self.u = u
        self.L = L
        self.CB = CB

    def search1(self, X, Y, v, d):
        """
        Perform the SEARCH1 algorithm.
        """
        if d == self.n + 1: # end of the tree
            self.CB = X # update the current best X solution (here there is not the check that the best solution is better than the current one. No problem, it there is one before calling the function search)
            self.L = v # update the current best value of X
        else:            
            if self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search1(amd.union_listOflists_list(X,self.O[d]), Y, self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u), d + 1)

            if self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search1(X,amd.union_listOflists_list(Y,self.O[d]), self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u), d + 1)          


def branch_and_bound_ds(O, Theta, p, o_XY, v_XY, g, u):
    start_time = time.time()  # Record the start time

    CB = None # initilize the current best solution
    L = float('-inf')
    
    SearchAlgorithm_instance = SearchAlgorithm(O, Theta, p, o_XY, v_XY, g, u, L, CB)
    SearchAlgorithm_instance.search1([], [], 0, 1)
    CB = SearchAlgorithm_instance.CB

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    return CB, round(elapsed_time,2)


### IDA*

In [3]:
class IDA_SearchAlgorithm:
    def __init__(self, O, Theta, p, o_XY, v_XY, g, u, L_initial_limit, CB = None, L_reduction = 0.1):
        """
        Initialize the IDA object.
        """
        self.O = O
        self.n = len(O) - 1 
        self.Theta = Theta
        self.p = p
        self.o_XY = o_XY
        self.v_XY = v_XY
        self.g = g
        self.u = u
        self.L = L_initial_limit
        self.CB = CB
        self.HF = -np.inf # different from the serach alg
        self.L_reduction = L_reduction # different from the serach alg

    def search2(self, X, Y, v, d):
        """
        Perform the SEARCH2 algorithm.
        """
        if d == self.n + 1: # end of the tree
            self.CB = X # update the current best X solution (here there is not the check that the best solution is better than the current one. No problem, it there is one before calling the function search)
            self.L = v # update the current best value of X 
        else:            
            if self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search2(amd.union_listOflists_list(X,self.O[d]), Y, self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u), d + 1)
            elif self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u) > self.HF:
                self.HF = self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u)

            if self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search2(X,amd.union_listOflists_list(Y,self.O[d]), self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u), d + 1)          
            elif self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) > self.HF:
                self.HF = self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) 


def ida(O, Theta, p, o_XY, v_XY, g, u, L_initial_limit ,L_reduction = 0.1):
    start_time = time.time()  # Record the start time

    CB = None # initilize the current best solution
    L = L_initial_limit
    
    while CB == None:
        HF = -np.inf
        IDA_SearchAlgorithm_instance = IDA_SearchAlgorithm(O, Theta, p, o_XY, v_XY, g, u, L, CB, L_reduction)
        IDA_SearchAlgorithm_instance.search2([], [], 0, 1)
        CB = IDA_SearchAlgorithm_instance.CB
        HF = IDA_SearchAlgorithm_instance.HF
        L = min(HF, L*(1-L_reduction))

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    return CB, round(elapsed_time,2)

### My implementation of an Evolutionary Algorithm to solve AMD (SLEA AMD)

In [4]:
# Define the fitness function V(X)
def fitness(X, Theta, p, g, u):
    return amd.v_X(X, Theta, p, g, u)

# Define how to decode a solution vector s into X
def decode_solution(s, O):
    return [O[i] for i in range(len(s)) if s[i] == 1]

# Initialize population of n individuals
def initialize_population(n, O):
    return [np.random.randint(2, size=len(O)) for _ in range(n)]

# Tournament selection
def tournament_selection(population, fitness_values, new_population_size, tournament_size=3):
    selected = []
    for _ in range(new_population_size):
        participants = random.sample(list(zip(population, fitness_values)), tournament_size)
        winner = max(participants, key=lambda x: x[1])
        selected.append(winner[0])
    return selected

# Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = np.concatenate([parent1[:point], parent2[point:]])
    child2 = np.concatenate([parent2[:point], parent1[point:]])
    return child1, child2

# Mutation
def mutate(individual, mutation_rate=0.01):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]  # Flip bit
    return individual

# Main evolutionary algorithm
def slea(O, n, new_population_size, tournament_size, max_evaluations, mutation_rate, Theta, p, g, u):
    start_time = time.time()  # Record the start time
    population = initialize_population(n, O)
    evaluations = 0
    
    while evaluations < max_evaluations:
        # Fitness assessment
        fitness_values = [fitness(decode_solution(ind, O), Theta, p, g, u) for ind in population]
        evaluations += len(population)
        
        # Tournament selection
        selected_individuals = tournament_selection(population, fitness_values, new_population_size, tournament_size)
        
        # Generate new individuals through crossover and mutation
        new_population = []
        while len(new_population) < n - new_population_size:
            parents = random.sample(selected_individuals, 2)
            child1, child2 = crossover(parents[0], parents[1])
            new_population.append(mutate(child1, mutation_rate))
            if len(new_population) < n - new_population_size:
                new_population.append(mutate(child2, mutation_rate))
        
        # Create new population
        population = selected_individuals + new_population[:n - new_population_size]

    # Final fitness assessment to get the best solution
    final_fitness_values = [fitness(decode_solution(ind, O), Theta, p, g, u) for ind in population]
    best_index = np.argmax(final_fitness_values)
    best_solution = population[best_index]

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    
    return best_solution, round(fitness(decode_solution(best_solution, O), Theta, p, g, u),2), round(elapsed_time,2)



## Applications

### Experiment 1 - No need of IR

An instance

In [5]:
# An instance

# Instance values 
number_of_outcomes = 30
number_of_types = 15
minimum_value = 0
maximum_value = 100

O = [[i] for i in range(number_of_outcomes)]
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value)
u_instance = amd.u_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value)

B&B DFS

In [6]:
beb_CB, beb_model_vX_value, beb_elapsed_time, beb_semi_random_CB, beb_semi_random_vX_value, beb_model_vs_rand = amd.find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, algorithm = branch_and_bound_ds)
beb_gForAllOutcomes_model, beb_uForAllOutcomes_model, beb_gForAllOutcomes_semiRandom, beb_uForAllOutcomes_semiRandom = amd.check_model_results(beb_CB, g_instance, u_instance, Theta, beb_semi_random_CB)

print("BEB data:")
print("Check IR for the model", amd.check_IR(Theta=Theta, X=beb_CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=beb_semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", beb_elapsed_time)
print("Model X =", beb_CB)
print("Semi random model X = ",beb_semi_random_CB)
print("\nModel vX value ", beb_model_vX_value)
print("Semi random model vX value ", beb_semi_random_vX_value)
print("Relative improvement of the model ", beb_model_vs_rand," %")

print("\n        ","g  ", "u")
print("model   ", round(np.mean(beb_gForAllOutcomes_model),2), round(np.mean(beb_uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(beb_gForAllOutcomes_semiRandom),2), round(np.mean(beb_uForAllOutcomes_semiRandom),2))

KeyboardInterrupt: 

IDA*

In [ ]:
# IDA* parameters
L_initial_limit = 80
L_reduction = 0.1

# Solve 
ida_CB, ida_model_vX_value, ida_elapsed_time, ida_semi_random_CB, ida_semi_random_vX_value, ida_model_vs_rand = amd.ida_find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, L_initial_limit, L_reduction, algorithm = ida)
ida_gForAllOutcomes_model, ida_uForAllOutcomes_model, ida_gForAllOutcomes_semiRandom, ida_uForAllOutcomes_semiRandom = amd.check_model_results(ida_CB, g_instance, u_instance, Theta, ida_semi_random_CB)

print("IDA data:")
print("Check IR for the model", amd.check_IR(Theta=Theta, X=ida_CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=ida_semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", ida_elapsed_time)
print("Model X =", ida_CB)
print("Semi random model X = ",ida_semi_random_CB)
print("\nModel vX value ", ida_model_vX_value)
print("Semi random model vX value ", ida_semi_random_vX_value)
print("Relative improvement of the model ", ida_model_vs_rand," %")

print("\n        ","g  ", "u")
print("model   ", round(np.mean(ida_gForAllOutcomes_model),2), round(np.mean(ida_uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(ida_gForAllOutcomes_semiRandom),2), round(np.mean(ida_uForAllOutcomes_semiRandom),2))

SLEA

In [ ]:
# Parameters
max_evaluations = 5000  # Maximum number of fitness evaluations
population_size = 30 # Number of individuals in the population
new_population_size = 10  # Number of individuals to generate in each generation
tournament_size = 3
mutation_rate = 0.1

# Solve 
best_solution, best_fitness, elapsed_time = slea(
    O=O,
    n=population_size,
    new_population_size=new_population_size,
    tournament_size=tournament_size,
    max_evaluations=max_evaluations,
    mutation_rate=0.1,
    Theta=Theta,
    p=p, 
    g=g_instance.g,
    u=u_instance.u
)
print("Best solution:", best_solution)
print("Best fitness slea:", best_fitness)
print("Best fitness beb:", beb_model_vX_value)
print("Best fitness ida:", ida_model_vX_value)
print("Elapsed time:", elapsed_time, "seconds\n")

X = decode_solution(best_solution, O)

print("Check IR for the model =", amd.check_IR(Theta=Theta, X=X, u=u_instance.u))
print("X = ", X)
print("Values of oX = ", best_fitness)

outcomeForAllThetas_model = [amd.o_X(theta = theta, X = X, g = g_instance.g, u = u_instance.u) for theta in Theta]
gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

print("\n        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))

### Experiment 2 - Need of IR

In [ ]:
# An instance
number_of_outcomes = 25
number_of_types = 15
minimum_value = -50
maximum_value = 50

O = [[i] for i in range(number_of_outcomes)]
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value)
u_instance = amd.u_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value)# Instance values 

In [ ]:
beb_CB, beb_model_vX_value, beb_elapsed_time, beb_semi_random_CB, beb_semi_random_vX_value, beb_model_vs_rand = amd.find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, algorithm = branch_and_bound_ds)
beb_gForAllOutcomes_model, beb_uForAllOutcomes_model, beb_gForAllOutcomes_semiRandom, beb_uForAllOutcomes_semiRandom = amd.check_model_results(beb_CB, g_instance, u_instance, Theta, beb_semi_random_CB)

print("BEB data:")
print("Check IR for the model", amd.check_IR(Theta=Theta, X=beb_CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=beb_semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", beb_elapsed_time)
print("Model X =", beb_CB)
print("Semi random model X = ",beb_semi_random_CB)
print("\nModel vX value ", beb_model_vX_value)
print("Semi random model vX value ", beb_semi_random_vX_value)
print("Relative improvement of the model ", beb_model_vs_rand," %")

print("\n        ","g  ", "u")
print("model   ", round(np.mean(beb_gForAllOutcomes_model),2), round(np.mean(beb_uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(beb_gForAllOutcomes_semiRandom),2), round(np.mean(beb_uForAllOutcomes_semiRandom),2))

In [ ]:
# IDA* parameters
L_initial_limit = 35
L_reduction = 0.1

# Solve 
ida_CB, ida_model_vX_value, ida_elapsed_time, ida_semi_random_CB, ida_semi_random_vX_value, ida_model_vs_rand = amd.ida_find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, L_initial_limit, L_reduction, algorithm = ida)
ida_gForAllOutcomes_model, ida_uForAllOutcomes_model, ida_gForAllOutcomes_semiRandom, ida_uForAllOutcomes_semiRandom = amd.check_model_results(ida_CB, g_instance, u_instance, Theta, ida_semi_random_CB)

print("IDA data:")
print("Check IR for the model", amd.check_IR(Theta=Theta, X=ida_CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=ida_semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", ida_elapsed_time)
print("Model X =", ida_CB)
print("Semi random model X = ",ida_semi_random_CB)
print("\nModel vX value ", ida_model_vX_value)
print("Semi random model vX value ", ida_semi_random_vX_value)
print("Relative improvement of the model ", ida_model_vs_rand," %")

print("\n        ","g  ", "u")
print("model   ", round(np.mean(ida_gForAllOutcomes_model),2), round(np.mean(ida_uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(ida_gForAllOutcomes_semiRandom),2), round(np.mean(ida_uForAllOutcomes_semiRandom),2))

In [ ]:
# Parameters
max_evaluations = 3000  # Maximum number of fitness evaluations
population_size = 30 # Number of individuals in the population
new_population_size = 10  # Number of individuals to generate in each generation
tournament_size = 3
mutation_rate = 0.1

# Solve 
best_solution, best_fitness, elapsed_time = slea(
    O=O,
    n=population_size,
    new_population_size=new_population_size,
    tournament_size=tournament_size,
    max_evaluations=max_evaluations,
    mutation_rate=0.1,
    Theta=Theta,
    p=p, 
    g=g_instance.g,
    u=u_instance.u
)
print("Best solution:", best_solution)
print("Best fitness slea:", best_fitness)
print("Best fitness beb:", beb_model_vX_value)
print("Best fitness ida:", ida_model_vX_value)
print("Elapsed time:", elapsed_time, "seconds\n")

X = decode_solution(best_solution, O)

print("Check IR for the model =", amd.check_IR(Theta=Theta, X=X, u=u_instance.u))
print("X = ", X)
print("Values of oX = ", best_fitness)

outcomeForAllThetas_model = [amd.o_X(theta = theta, X = X, g = g_instance.g, u = u_instance.u) for theta in Theta]
gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

print("\n        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))

### Experiment 3 - Bartering problem

In [ ]:
# An instance
number_of_items = 10
number_of_outcomes = 25
number_of_types = 15
minimum_value = 0 # try -5
maximum_value = 10 # try 5

all_possible_outcomes = list(itertools.product([0, 1], repeat=number_of_items))
O = random.sample(all_possible_outcomes, number_of_outcomes)
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_bartering_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value)
u_instance = amd.u_bartering_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value) 



In [ ]:
beb_CB, beb_model_vX_value, beb_elapsed_time, beb_semi_random_CB, beb_semi_random_vX_value, beb_model_vs_rand = amd.find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, algorithm = branch_and_bound_ds)
beb_gForAllOutcomes_model, beb_uForAllOutcomes_model, beb_gForAllOutcomes_semiRandom, beb_uForAllOutcomes_semiRandom = amd.check_model_results(beb_CB, g_instance, u_instance, Theta, beb_semi_random_CB)

print("BEB data:")
print("Check IR for the model", amd.check_IR(Theta=Theta, X=beb_CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=beb_semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", beb_elapsed_time)
print("Model X =", beb_CB)
print("Semi random model X = ",beb_semi_random_CB)
print("\nModel vX value ", beb_model_vX_value)
print("Semi random model vX value ", beb_semi_random_vX_value)
print("Relative improvement of the model ", beb_model_vs_rand," %")

print("\n        ","g  ", "u")
print("model   ", round(np.mean(beb_gForAllOutcomes_model),2), round(np.mean(beb_uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(beb_gForAllOutcomes_semiRandom),2), round(np.mean(beb_uForAllOutcomes_semiRandom),2))

In [ ]:
# IDA* parameters
L_initial_limit = 50
L_reduction = 0.1

# Solve 
ida_CB, ida_model_vX_value, ida_elapsed_time, ida_semi_random_CB, ida_semi_random_vX_value, ida_model_vs_rand = amd.ida_find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, L_initial_limit, L_reduction, algorithm = ida)
ida_gForAllOutcomes_model, ida_uForAllOutcomes_model, ida_gForAllOutcomes_semiRandom, ida_uForAllOutcomes_semiRandom = amd.check_model_results(ida_CB, g_instance, u_instance, Theta, ida_semi_random_CB)

print("IDA data:")
print("Check IR for the model", amd.check_IR(Theta=Theta, X=ida_CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=ida_semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", ida_elapsed_time)
print("Model X =", ida_CB)
print("Semi random model X = ",ida_semi_random_CB)
print("\nModel vX value ", ida_model_vX_value)
print("Semi random model vX value ", ida_semi_random_vX_value)
print("Relative improvement of the model ", ida_model_vs_rand," %")

print("\n        ","g  ", "u")
print("model   ", round(np.mean(ida_gForAllOutcomes_model),2), round(np.mean(ida_uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(ida_gForAllOutcomes_semiRandom),2), round(np.mean(ida_uForAllOutcomes_semiRandom),2))

In [ ]:
# Parameters
max_evaluations = 10000  # Maximum number of fitness evaluations
population_size = 30 # Number of individuals in the population
new_population_size = 10  # Number of individuals to generate in each generation
tournament_size = 10
mutation_rate = 0.1

# Solve 
best_solution, best_fitness, elapsed_time = slea(
    O=O,
    n=population_size,
    new_population_size=new_population_size,
    tournament_size=tournament_size,
    max_evaluations=max_evaluations,
    mutation_rate=0.1,
    Theta=Theta,
    p=p, 
    g=g_instance.g,
    u=u_instance.u
)
print("Best solution:", best_solution)
print("Best fitness slea:", best_fitness)
print("Best fitness beb:", beb_model_vX_value)
print("Best fitness ida:", ida_model_vX_value)
print("Elapsed time:", elapsed_time, "seconds\n")

X = decode_solution(best_solution, O)

print("Check IR for the model =", amd.check_IR(Theta=Theta, X=X, u=u_instance.u))
print("X = ", X)
print("Values of oX = ", best_fitness)

outcomeForAllThetas_model = [amd.o_X(theta = theta, X = X, g = g_instance.g, u = u_instance.u) for theta in Theta]
gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

print("\n        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))

## Extra bartering 

In [41]:
def initialize_goods():
    # Create an array with 3 elements of 0 and 3 elements of 1
    initial_goods = [0] * 3 + [1] * 3
    
    # Shuffle the array to mix the elements
    random.shuffle(initial_goods)
    
    return initial_goods

In [50]:
# An instance
number_of_items = 6
number_of_outcomes = 30
number_of_types = 5
minimum_value = 1 # with [1,10] the IR constraint is mainly = False 
maximum_value = 5

all_possible_outcomes = list(itertools.product([0, 1], repeat=number_of_items))
O = random.sample(all_possible_outcomes, number_of_outcomes)
initial_goods = initialize_goods()
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_bartering_class_difference(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value, initial_goods=initial_goods)
u_instance = amd.u_bartering_class_difference(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value, initial_goods=initial_goods)  



In [51]:
beb_CB, beb_model_vX_value, beb_elapsed_time, beb_semi_random_CB, beb_semi_random_vX_value, beb_model_vs_rand = amd.find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, algorithm = branch_and_bound_ds)
beb_gForAllOutcomes_model, beb_uForAllOutcomes_model, beb_gForAllOutcomes_semiRandom, beb_uForAllOutcomes_semiRandom = amd.check_model_results(beb_CB, g_instance, u_instance, Theta, beb_semi_random_CB)

print("BEB data:")
print("Check IR for the model", amd.check_IR(Theta=Theta, X=beb_CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=beb_semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", beb_elapsed_time)
print("Model X =", beb_CB)
print("Semi random model X = ",beb_semi_random_CB)
print("\nModel vX value ", beb_model_vX_value)
print("Semi random model vX value ", beb_semi_random_vX_value)
print("Relative improvement of the model ", beb_model_vs_rand," %")

print("\n        ","g  ", "u")
print("model   ", round(np.mean(beb_gForAllOutcomes_model),2), round(np.mean(beb_uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(beb_gForAllOutcomes_semiRandom),2), round(np.mean(beb_uForAllOutcomes_semiRandom),2))

BEB data:
Check IR for the model False
Check IR for the semi-rnd True

Elapsed time =  6.15
Model X = [[0, 0, 0, 0, 0, 1]]
Semi random model X =  [(1, 0, 0, 1, 0, 1)]

Model vX value  10.0
Semi random model vX value  2.0
Relative improvement of the model  80.0  %

         g   u
model    10.0 -7.8
semi rnd 2.0 -0.4


In [47]:
# IDA* parameters
L_initial_limit = 2
L_reduction = 0.1

# Solve 
ida_CB, ida_model_vX_value, ida_elapsed_time, ida_semi_random_CB, ida_semi_random_vX_value, ida_model_vs_rand = amd.ida_find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, L_initial_limit, L_reduction, algorithm = ida)
ida_gForAllOutcomes_model, ida_uForAllOutcomes_model, ida_gForAllOutcomes_semiRandom, ida_uForAllOutcomes_semiRandom = amd.check_model_results(ida_CB, g_instance, u_instance, Theta, ida_semi_random_CB)

print("IDA data:")
print("Check IR for the model", amd.check_IR(Theta=Theta, X=ida_CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=ida_semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", ida_elapsed_time)
print("Model X =", ida_CB)
print("Semi random model X = ",ida_semi_random_CB)
print("\nModel vX value ", ida_model_vX_value)
print("Semi random model vX value ", ida_semi_random_vX_value)
print("Relative improvement of the model ", ida_model_vs_rand," %")

print("\n        ","g  ", "u")
print("model   ", round(np.mean(ida_gForAllOutcomes_model),2), round(np.mean(ida_uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(ida_gForAllOutcomes_semiRandom),2), round(np.mean(ida_uForAllOutcomes_semiRandom),2))

KeyboardInterrupt: 

In [37]:
# Parameters
max_evaluations = 2000  # Maximum number of fitness evaluations
population_size = 30 # Number of individuals in the population
new_population_size = 10  # Number of individuals to generate in each generation
tournament_size = 3
mutation_rate = 0.1

# Solve 
best_solution, best_fitness, elapsed_time = slea(
    O=O,
    n=population_size,
    new_population_size=new_population_size,
    tournament_size=tournament_size,
    max_evaluations=max_evaluations,
    mutation_rate=mutation_rate,
    Theta=Theta,
    p=p, 
    g=g_instance.g,
    u=u_instance.u
)
print("Best solution:", best_solution)
print("Best fitness slea:", best_fitness)
#print("Best fitness beb:", beb_model_vX_value)
print("Best fitness ida:", ida_model_vX_value)
print("Elapsed time:", elapsed_time, "seconds\n")

X = decode_solution(best_solution, O)

print("Check IR for the model =", amd.check_IR(Theta=Theta, X=X, u=u_instance.u))
print("X = ", X)
print("Values of oX = ", best_fitness)

outcomeForAllThetas_model = [amd.o_X(theta = theta, X = X, g = g_instance.g, u = u_instance.u) for theta in Theta]
gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

print("\n        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))

Best solution: [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1 0
 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1]
Best fitness slea: -3.6
Best fitness ida: 1.0
Elapsed time: 9.09 seconds

Check IR for the model = False
X =  [(0, 1, 1, 0, 0, 1), (0, 1, 1, 1, 0, 0), (0, 0, 0, 1, 1, 1), (0, 1, 0, 1, 0, 1), (0, 0, 0, 0, 1, 0), (0, 1, 1, 0, 1, 0), (0, 0, 1, 0, 0, 1), (1, 0, 1, 0, 1, 0), (1, 0, 0, 0, 0, 0), (1, 0, 1, 1, 0, 0), (1, 1, 1, 1, 0, 0), (1, 0, 0, 0, 0, 1), (0, 0, 1, 1, 1, 0), (1, 0, 0, 1, 1, 1), (0, 0, 1, 0, 0, 0), (1, 0, 0, 1, 0, 1), (0, 0, 0, 1, 1, 0), (1, 0, 0, 1, 1, 0), (1, 0, 1, 0, 0, 0), (0, 1, 1, 1, 1, 0), (1, 1, 0, 0, 0, 0)]
Values of oX =  -3.6

         g   u
model    -3.6 6.4
